In [1]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=0a074fcc49ae176df4c60169f76f8e3104626d093cba17978addc53eb8851b29
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [2]:
import pandas as pd
import numpy as np
import re
import os
import unicodedata
import html
from langdetect import detect, DetectorFactory
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from tqdm.notebook import tqdm

# --- 1. CẤU HÌNH BAN ĐẦU ---
# Paths
DATA_DIR = "/kaggle/input/vlsp-medical"
SRC_FILE = os.path.join(DATA_DIR, "train.en.txt")
TGT_FILE = os.path.join(DATA_DIR, "train.vi.txt")
FINAL_DATA_PATH = "vlsp_medical_cleaned_final.csv"
TOKENIZER_DIR = "tokenizer_files"

# Regex
URL_REGEX = r'(?:https?://|www\.)\S+'
CONTROL_CHAR_REGEX = r'[\x00-\x1F\x7F-\x9F]' # Ký tự control
VN_CHAR_REGEX = re.compile(r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]')

# Cố định seed
DetectorFactory.seed = 0
tqdm.pandas()

In [3]:
def preprocess_text(text):
    if not isinstance(text, str): return ""
    
    # 1. Giải mã HTML entities
    text = html.unescape(text)
    
    # 2. Xóa HTML Tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # 3. Xóa URL triệt để
    text = re.sub(URL_REGEX, ' ', text)
    
    # 4. Xóa Ký tự Control không in được
    text = re.sub(CONTROL_CHAR_REGEX, ' ', text)
    
    # 5. Chuẩn hóa Unicode về dạng NFC (QUAN TRỌNG cho tiếng Việt)
    text = unicodedata.normalize('NFC', text)
    
    # 6. Chuẩn hóa/Xóa các loại dấu ngoặc đặc biệt thường là noise
    text = re.sub(r'[【】「」『』“”‘’“”]', '', text)
    
    # 7. Xóa ngoặc đơn () nếu chúng không cân bằng
    if text.count('(') != text.count(')'):
        text = text.replace('(', '').replace(')', '')
        
    # 8. Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_vietnamese_text(text):
    # Kiểm tra Tiếng Việt trong cột En (dùng regex nhanh hơn langdetect)
    text = str(text).lower()
    return bool(VN_CHAR_REGEX.search(text))

def detect_safe(text):
    # Langdetect an toàn
    try:
        return detect(text) if len(str(text)) > 3 else 'unknown'
    except: return 'unknown'

def is_valid_ratio_and_len(row):
    en_l = row['en_len_check']
    vi_l = row['vi_len_check']
    ratio = row['len_ratio']   # ratio = vi/en

    # 1. Lọc câu quá ngắn (min 3 từ)
    if en_l < 3 or vi_l < 3:
        return False

    # 2. Lọc câu quá dài
    if en_l > 128 or vi_l > 256:
        return False

    # 3. Lọc trường hợp tiếng Việt ngắn bất thường
    if ratio < 0.5:
        return False

    # 4. Ngưỡng theo độ dài câu tiếng Anh
    if en_l < 5:          # câu rất ngắn
        return ratio <= 5.0
    if en_l < 12:         # câu ngắn - vừa
        return ratio <= 4.0

    # Câu dài
    return ratio <= 3.5

def train_tokenizer_from_list(text_list, vocab_size=32000):
    tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"],
    )
    tokenizer.train_from_iterator(text_list, trainer)
    return tokenizer

In [4]:
# --- 3. PIPELINE EXECUTION ---

print("1. Đang Load và ÁP DỤNG PREPROCESSING LẦN 1 (Xóa URL/Control Chars)...")
with open(SRC_FILE, 'r', encoding='utf-8') as f: 
    src_lines = [preprocess_text(line) for line in f.read().splitlines()]
with open(TGT_FILE, 'r', encoding='utf-8') as f: 
    tgt_lines = [preprocess_text(line) for line in f.read().splitlines()]

df = pd.DataFrame({'en': src_lines, 'vi': tgt_lines})

# Xóa các dòng rỗng tạo ra sau khi làm sạch
df = df[(df['en'].str.len() > 0) & (df['vi'].str.len() > 0)].copy()

print(f"   -> Số lượng sau khi làm sạch sơ bộ: {len(df)}")


print("2. LỌC: Xóa Trùng Lặp & Tiếng Việt trong cột En...")
# Lọc 1: Xóa trùng lặp (100% match)
df_clean = df.drop_duplicates().copy()

# Lọc 2: Xóa Tiếng Việt trong cột En (dùng regex nhanh)
df_clean = df_clean[~df_clean['en'].apply(is_vietnamese_text)]
print(f"   -> Số lượng sau khi xóa trùng và lọc VN trong En: {len(df_clean)}")


print("3. LỌC: LangDetect, Ratio và Độ dài...")
# Tính độ dài & Ratio
df_clean['en_len_check'] = df_clean['en'].str.split().str.len().fillna(0)
df_clean['vi_len_check'] = df_clean['vi'].str.split().str.len().fillna(0)
df_clean['len_ratio'] = np.where(df_clean['en_len_check'] > 0, df_clean['vi_len_check'] / df_clean['en_len_check'], 0)

# LangDetect (Tốn thời gian)
df_clean['en_detected'] = df_clean['en'].progress_apply(detect_safe)

# Lọc 3: Xóa LangDetect (En bị detect là Vi)
garbage_mask = (df_clean['en_detected'] == 'vi') & (df_clean['en_len_check'] > 3)
df_final = df_clean[~garbage_mask].copy()

# Lọc 4: Áp dụng Ratio và Length (Min 4 từ)
df_final = df_final[df_final.apply(is_valid_ratio_and_len, axis=1)].copy()

# Chuyển list cho Tokenizer
train_src_list = df_final['en'].tolist()
train_tgt_list = df_final['vi'].tolist()


print(f"\n4. CHỐT SỔ: SỐ LƯỢNG CẶP SẠCH CUỐI CÙNG: {len(train_src_list)}")
print("   -> Bắt đầu train Tokenizer...")

1. Đang Load và ÁP DỤNG PREPROCESSING LẦN 1 (Xóa URL/Control Chars)...
   -> Số lượng sau khi làm sạch sơ bộ: 500000
2. LỌC: Xóa Trùng Lặp & Tiếng Việt trong cột En...
   -> Số lượng sau khi xóa trùng và lọc VN trong En: 346379
3. LỌC: LangDetect, Ratio và Độ dài...


  0%|          | 0/346379 [00:00<?, ?it/s]


4. CHỐT SỔ: SỐ LƯỢNG CẶP SẠCH CUỐI CÙNG: 344426
   -> Bắt đầu train Tokenizer...


In [5]:
# --- 4. TRAIN TOKENIZER & LƯU TRỮ ---

# Train Tokenizer
src_tokenizer = train_tokenizer_from_list(train_src_list)
tgt_tokenizer = train_tokenizer_from_list(train_tgt_list)

print(f"   -> Vocab En: {src_tokenizer.get_vocab_size()}, Vocab Vi: {tgt_tokenizer.get_vocab_size()}")

# LƯU TRỮ FINAL
os.makedirs(TOKENIZER_DIR, exist_ok=True)
df_final[['en', 'vi']].to_csv(FINAL_DATA_PATH, index=False, encoding='utf-8')
src_tokenizer.save(os.path.join(TOKENIZER_DIR, "en_tokenizer.json"))
tgt_tokenizer.save(os.path.join(TOKENIZER_DIR, "vi_tokenizer.json"))

print("\n=======================================================")
print(f"🎉 HOÀN TẤT NOTEBOOK 1! DỮ LIỆU ĐÃ SẠCH TUYỆT ĐỐI! 🎉")
print(f"   Dữ liệu sạch đã lưu tại: {FINAL_DATA_PATH}")
print(f"   Tokenizer đã lưu tại: {TOKENIZER_DIR}/...")
print("=======================================================")







   -> Vocab En: 32000, Vocab Vi: 32000

🎉 HOÀN TẤT NOTEBOOK 1! DỮ LIỆU ĐÃ SẠCH TUYỆT ĐỐI! 🎉
   Dữ liệu sạch đã lưu tại: vlsp_medical_cleaned_final.csv
   Tokenizer đã lưu tại: tokenizer_files/...


In [6]:
# @title 5. KIỂM TRA VÀ LÀM SẠCH LẦN CUỐI (QC CHỐT SỔ)
# Sử dụng df_final vừa được tạo ra để đảm bảo dữ liệu mới nhất
df_qc = df_final[['en', 'vi']].copy()
initial_count = len(df_qc)

# --- A. XÓA TRÙNG LẶP LẦN CUỐI (TRÁNH LỖI PHÁT SINH KHI LÀM SẠCH) ---
df_qc.drop_duplicates(inplace=True)
duplicates_removed = initial_count - len(df_qc)

# --- B. KIỂM TRA RATIO CỰC ĐOAN (VI/EN > 5.0) ---
df_qc['en_len_qc'] = df_qc['en'].str.split().str.len().fillna(0)
df_qc['vi_len_qc'] = df_qc['vi'].str.split().str.len().fillna(0)
df_qc['len_ratio_qc'] = np.where(df_qc['en_len_qc'] > 0, df_qc['vi_len_qc'] / df_qc['en_len_qc'], 0)

extreme_ratio_filter = (df_qc['len_ratio_qc'] > 5.0)
num_extreme_ratio = extreme_ratio_filter.sum()

if num_extreme_ratio > 0:
    print(f"🚩 PHÁT HIỆN LỖI: {num_extreme_ratio} dòng có tỷ lệ Vi/En > 5.0. Đang loại bỏ...")
    df_qc = df_qc[~extreme_ratio_filter]
    
# --- C. BÁO CÁO & CẬP NHẬT LIST ---
removed_total = initial_count - len(df_qc)
final_clean_count = len(df_qc)

print("\n--- KẾT QUẢ QC CUỐI CÙNG ---")
print(f"✅ Đã xóa {duplicates_removed} dòng bị trùng lặp.")
print(f"✅ Đã xóa {num_extreme_ratio} dòng có Ratio > 5.0.")
print(f"TỔNG SỐ DÒNG ĐÃ XÓA TRONG QC: {removed_total}")
print(f"SỐ LƯỢNG DỮ LIỆU SẠCH TUYỆT ĐỐI: {final_clean_count} dòng.")

# CẬP NHẬT LẠI LIST CUỐI CÙNG (QUAN TRỌNG)
train_src_list = df_qc['en'].tolist()
train_tgt_list = df_qc['vi'].tolist()

# Đảm bảo df_final được cập nhật để lưu
df_final = df_qc[['en', 'vi']]


--- KẾT QUẢ QC CUỐI CÙNG ---
✅ Đã xóa 0 dòng bị trùng lặp.
✅ Đã xóa 0 dòng có Ratio > 5.0.
TỔNG SỐ DÒNG ĐÃ XÓA TRONG QC: 0
SỐ LƯỢNG DỮ LIỆU SẠCH TUYỆT ĐỐI: 344426 dòng.
